In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import subplot
from tqdm import tqdm
import h5py 
import os
import re
import subprocess
import pandas as pd

In [ ]:
folder_path = "../"
write_path = "../gwtc3_converted/"

PLANCK_2018_Ho = 2.1816926176539463e-18 ### CGS
PLANCK_2018_OmegaMatter = 0.3158
PLANCK_2018_OmegaLambda = 1. - PLANCK_2018_OmegaMatter
PLANCK_2018_OmegaRadiation = 0.

In [ ]:
# Read the data
files = (file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file)))
events_files = list(filter(lambda x: x.endswith((".h5", ".hdf5")), files))
for i in range(len(events_files)):print(f"{i+1}. {events_files[i]} (i = {i})")

In [ ]:
if not os.path.exists(write_path + "event-list.txt"):
    with open(write_path + "event-list.txt", "w") as f:
        pass

In [ ]:
#regex
re1 = r"GWTC-1"
re2= r"GWTC2p1"
re3 = r"GWTC3"

In [ ]:
event_waveforms = []
compile_df = pd.DataFrame(columns=["event", "waveform", "cosmology", "prior_cosmology1", "prior_cosmology2", "prior_a_1 (or chi1)", "prior_a_2 (or chi2)", "prior_tilt_1", "prior_tilt_2"])


for i in tqdm(range(len(events_files))): 
    file_path = events_files[i]
    event_file = folder_path+file_path
    with h5py.File(event_file,'r+') as file: 
        wave_index = 0
        if i in [14]:
            wave_index = 2
        waveform = list(file.keys())[wave_index]
        # print(file_path)
        # print(file.keys())
        # print(file[waveform].dtype.descr)
        # print(file[waveform]["priors"].keys())
        # print(file[waveform]["meta_data"]['meta_data'].keys())
        # print(file[waveform]["description"][:])
        # print(file[waveform]["priors"]["analytic"].keys())
        # print(file[waveform]["meta_data"]['meta_data'].keys())
        # print(file[waveform]["priors"]['analytic']["luminosity_distance"][:][0])
        # print(file[waveform]["priors"]['analytic']["a_1"][:][0])
        # print(compile_df.columns)
        # print(list_row)
        # print(file[waveform]["posterior_samples"].dtype.descr)

        if i in [13, 23, 25, 41, 46, 61]:
            spin1_used = "NOT INCLUDED (Uniform)" ; spin2_used = "NOT INCLUDED (Uniform)"      
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif i == 42: # GW190425 - https://arxiv.org/pdf/2108.01045
            spin1_used = "chi < 0.89"; spin2_used = "chi < 0.89"   
            tilt1_used = "NOT INCLUDED (Isotropic)" ; tilt2_used = "NOT INCLUDED (Isotropic)"
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED"
        elif i == 48: # GW170817 - https://arxiv.org/pdf/1805.11579
            # print(file["IMRPhenomPv2NRT_highSpin_prior"]["costilt1"])
            # print(file[waveform].dtype.descr)
            cosmology_used = "planck 2015"
            cosmo_prior = "NOT INCLUDED"
            spin1_used = "chi < 0.89"; spin2_used = "chi < 0.89"
            tilt1_used = "NOT INCLUDED (Isotropic)"; tilt2_used = "NOT INCLUDED (Isotropic)"
        else:
            cosmo_prior = file[waveform]["priors"]['analytic']["luminosity_distance"][:][0]
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            spin1_used = str(file[waveform]["priors"]['analytic']["a_1"][:][0]) ; spin2_used = str(file[waveform]["priors"]['analytic']["a_2"][:][0])
            tilt1_used = str(file[waveform]["priors"]['analytic']["tilt_1"][:][0]) ; tilt2_used = str(file[waveform]["priors"]['analytic']["tilt_2"][:][0])
            
        list_row = [events_files[i], waveform, cosmology_used, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
        event_waveforms.append(f"Converting {waveform} from {event_file}")
                                               
            
    args = [f"{waveform}", f"{event_file}",
            "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
            f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}"]
    
    if re.search(re1, file_path):
        cosmo_cat = "d_l^2"  # According to Amanda's O4a work
        subprocess.run(["python", "h52csv_GWTC1.py", *args])
    elif re.search(re2, file_path):
        cosmo_cat = "uniform in comoving volume"
        subprocess.run(["python", "h52csv_GWTC2p1.py", *args])
    elif re.search(re3, file_path):
        cosmo_cat = "uniform in comoving volume"
        subprocess.run(["python", "h52csv_GWTC3.py", *args])
    else:
        raise ValueError("No match found")
    
    list_row = [events_files[i], waveform, cosmology_used, cosmo_cat, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
    compile_df.loc[i] = list_row
    
compile_df.to_csv("compile_df.csv", index=False)
compile_df

In [ ]:
# read txt file
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.readlines()

print("Number of events converted: ", len(event_txt_files))
print("Total number of events: ", len(events_files))

In [ ]:
# Read events and confirm the conversion. 
import numpy as np
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.read().splitlines()
event_txt_files

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

for i in tqdm(range(len(event_txt_files))):
    event = event_txt_files[i]
    df = pd.DataFrame(np.genfromtxt(write_path + event, delimiter=",", names=True))
    fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(10, 4))
    ax[0, 0].hist(df["mass1_source"]); ax[0, 0].set_xlabel("mass1_source")
    ax[0, 1].hist(df["mass2_source"]); ax[0, 1].set_xlabel("mass2_source")
    ax[0, 2].hist(df["luminosity_distance"]); ax[0, 2].set_xlabel("luminosity_distance")
    ax[0, 3].hist(df["redshift"]); ax[0, 3].set_xlabel("redshift")
    ax[1, 0].hist(df["a_1"]); ax[1, 0].set_xlabel("a_1")
    ax[1, 1].hist(df["a_2"]); ax[1, 1].set_xlabel("a_2")
    ax[1, 2].hist(df["costilt1"]); ax[1, 2].set_xlabel("costilt1")
    ax[1, 3].hist(df["costilt2"]); ax[1, 3].set_xlabel("costilt1")
    plt.suptitle(event[:8] + f" - PE Size: {df.shape[0]}")
    plt.tight_layout()
    plt.savefig("verify_PE/" + event[:8] + ".png")
    plt.close()
    

In [ ]:
# i = 48
# event = event_txt_files[i]
# print(event)
# df = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[i], delimiter=",", names=True))
# df_normal = pd.DataFrame(np.genfromtxt(write_path + event_txt_files[50], delimiter=",", names=True))
# print(df.columns)
# print(df_normal.columns) 
#Additionally includes geocenter_time, lnprob_geocenter_time, lnprob_spin1x_spin1y_spin1z, lnprob_spin2x_spin2y_spin2z, loglikelihood, luminosity_distance, spin1x, spin1y, spin1z, spin2x, spin2y, spin2z. (This has changed now but was true before modification of h52csv_GWTC1.py)

# Conversion of Injections

In [ ]:
vt_folder = "../../vt/"
vt_file = "o1+o2+o3_mixture_real+semianalytic-LIGO-T2100377-v2.hdf5"
vt_file_O3_full = "endo3_mixture-LIGO-T2100113-v12.hdf5"
vt_path = vt_folder + vt_file

In [ ]:
with h5py.File(vt_folder + vt_file_O3_full, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key])) 
    data = list(f[a_group_key])
    print(data)
    
with h5py.File(vt_path, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key])) 
    data = list(f[a_group_key])
    print(data)

In [ ]:
with h5py.File(vt_path,'r+') as file: 
        waveform = list(file.keys())[0]
        event_waveforms.append(f"Converting {waveform} from {vt_path}")
        
args = ["--max-far", f"{1}",
        "--Ho", f"{PLANCK_2018_Ho}", 
        "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", 
        "--OmegaRadiation", f"{PLANCK_2018_OmegaRadiation}", 
        "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", 
        f"{vt_path}",
        # "-v", 
        # f"{vt_folder}" + "sensitivity-estimate_O3.csv.gz"]
        f"{vt_folder}" + "sensitivity-estimate.csv.gz"]

subprocess.run(["python", "h52csv_injections.py", *args])
print("Done")

In [ ]:
path = f"{vt_folder}" + "sensitivity-estimate.csv.gz"
df = pd.read_csv(path)
df # Checking injections